# Customer Interest Clustering for Marketing

This is a tutorial on how to perform customer clustering based on the interests and purcahses of customers. 

Marketing teams want to know who how to appeal to their customers.

We'll show how using graph analytics can be used to gain insights about the interests of customers by finding communities of customers who've bought similar products. 

We'lll accomplish this by creating a bipartite graph of customers and products, using a graph projection to create a graph of customers linked to other customers who've bought the same product, and using Louvain community detection to find the communities.

We'll be using ecommerce transaction data from a U.K. retailer provided by the University of California, Irvine. The data can be found [here](https://www.kaggle.com/carrie1/ecommerce-data).

# Data Preprocessing

Let's first look at the data.

First, we'll need to import some libraries.

In [1]:
import metagraph as mg
import pandas as pd
import networkx as nx

Let's see what the data looks like.

In [2]:
RAW_DATA_CSV = './data.csv' # https://www.kaggle.com/carrie1/ecommerce-data
data_df = pd.read_csv(RAW_DATA_CSV, encoding="ISO-8859-1")
data_df.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom

Let's clean the data to make sure there aren't any missing values. 

In [3]:
data_df.InvoiceDate = pd.to_datetime(data_df.InvoiceDate, format="%m/%d/%Y %H:%M")
data_df.drop(data_df.index[data_df.CustomerID != data_df.CustomerID], inplace=True)
assert len(data_df[data_df.isnull().any(axis=1)])==0, "Raw data contains NaN"
data_df = data_df.astype({'CustomerID': int}, copy=False)
data_df.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55       17850  United Kingdom  
1 2010-12-01 08:26:00       3.39       17850  United Kingdom  
2 2010-12-01 08:26:00       2.75       17850  United Kingdom  
3 2010-12-01 08:26:00       3.39       17850  United Kingdom  
4 2010-12-01 08:26:00       3.39       17850  United Kingdom

Note that some of these transactions are for returns (denoted by negative quantity values).

In [4]:
data_df[data_df.Quantity < 1].head()

InvoiceNo StockCode                       Description  Quantity  \
141   C536379         D                          Discount        -1   
154   C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1   
235   C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12   
236   C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24   
237   C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24   

            InvoiceDate  UnitPrice  CustomerID         Country  
141 2010-12-01 09:41:00      27.50       14527  United Kingdom  
154 2010-12-01 09:49:00       4.65       15311  United Kingdom  
235 2010-12-01 10:24:00       1.65       17548  United Kingdom  
236 2010-12-01 10:24:00       0.29       17548  United Kingdom  
237 2010-12-01 10:24:00       0.29       17548  United Kingdom

Though customers may have returned these products, they did initially purchase the products (which reflects an interest in the product), so we'll keep the intiial purchases. However, we'll remove the return transactions (which will also remove any discount transactions as well). 

In [5]:
data_df.drop(data_df.index[data_df.Quantity <= 0], inplace=True)
data_df.head()

InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55       17850  United Kingdom  
1 2010-12-01 08:26:00       3.39       17850  United Kingdom  
2 2010-12-01 08:26:00       2.75       17850  United Kingdom  
3 2010-12-01 08:26:00       3.39       17850  United Kingdom  
4 2010-12-01 08:26:00       3.39       17850  United Kingdom

# Community Detection

Let's now find the communities of customers with similar purchases / interests. 

First, we'll need to create a bipartite graph of customers and products. 

In [6]:
bipartite_graph = nx.from_pandas_edgelist(data_df, 'CustomerID', 'StockCode')

Next, we'll need to use a graph projection to create a graph of customers linked to other customers who've bought the same product.

In [7]:
customer_similarity_graph = nx.projected_graph(bipartite_graph, data_df.CustomerID.unique())

Now, we'll need to use Louvain community detection to find customer similarity communities. 

In [8]:
r = mg.resolver
customer_similarity_graph_wrapped = r.wrappers.EdgeMap.NetworkXEdgeMap(customer_similarity_graph)
labels, modularity_score = r.algos.clustering.louvain_community(customer_similarity_graph_wrapped)

Let's now merge the labels into our dataframe.

In [9]:
data_df['CustomerCommunityLabel'] = data_df.CustomerID.map(lambda customer_id: labels.value[customer_id])
data_df.sample(10)

InvoiceNo StockCode                         Description  Quantity  \
300254    563194     84356                      POMPOM CURTAIN         2   
268444    560422     22356         CHARLOTTE BAG PINK POLKADOT        10   
309586    564096     23298                      SPOTTY BUNTING        50   
380702    569832     22336       DOVE DECORATION PAINTED ZINC         96   
291214    562437     23182      TOILET SIGN OCCUPIED OR VACANT        24   
282231    561631     23171            REGENCY TEA PLATE GREEN         12   
296772    562932     20726                  LUNCH BAG WOODLAND         2   
129236    547380     21977  PACK OF 60 PINK PAISLEY CAKE CASES         6   
338071    566495     21983    PACK OF 12 BLUE PAISLEY TISSUES          2   
131860    547650     21499                  BLUE POLKADOT WRAP        25   

               InvoiceDate  UnitPrice  CustomerID         Country  \
300254 2011-08-12 16:36:00       6.95       17858  United Kingdom   
268444 2011-07-18 15:14:00       0.85       16746  United Kingdom   
309586 2011-08-23 10:36:00       4.15       16133  United Kingdom   
380702 2011-10-06 12:51:00       0.19       15315  United Kingdom   
291214 2011-08-04 17:58:00       0.83       13047  United Kingdom   
282231 2011-07-28 14:27:00       1.65       14911            EIRE   
296772 2011-08-10 16:39:00       1.65       16904  United Kingdom   
129236 2011-03-22 15:20:00       0.55       16809  United Kingdom   
338071 2011-09-13 11:14:00       0.39       15529  United Kingdom   
131860 2011-03-24 12:08:00       0.42       17059  United Kingdom   

        CustomerCommunityLabel  
300254                       0  
268444                       1  
309586                       0  
380702                       0  
291214                       0  
282231                       2  
296772                       2  
129236                       1  
338071                       1  
131860                       1

We now have clusters of customers who've bought similar products and can market to these interests. 